#### Test of stuff exported from Mathematica for case "MatrixElementExport" of CryoTriple4B4W4B4W4B4W

## Preliminaries

NOTE: Edit the hardcoded path in the cell below to point to where CryoTriple4B4W4B4W4B4W_MatrixElementExport_triplep.py and symbexportCryoTriple4B4W4B4W4B4Wfull.py are located

In [4]:
import sys
sys.path.append('/Users/mbarton/Work/LIGO/analysis/AdvLIGOModel/GCIF Models/CryoTriple4B4W4B4W4B4W/MatrixElementExport/MatlabPythonMatrixElementsCalc')

### Other system dependencies

In [6]:
import os
import pandas as pd
import inspect
from math import pi,sqrt
from decimal import Decimal

import numpy as np

# import scipy.linalg
from matplotlib.ticker import MultipleLocator
import control as ct

import matplotlib.pyplot as plt
import importlib

import matplotlib
matplotlib.rcParams['font.sans-serif'] = "Comic Sans MS"
# Then, "ALWAYS use sans-serif fonts"
matplotlib.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.size'] = 18          # Default font size for text
plt.rcParams['axes.titlesize'] = 22     # Font size for axes titles
plt.rcParams['axes.labelsize'] = 18     # Font size for axes labels
plt.rcParams['xtick.labelsize'] = 14    # Font size for x-axis tick labels
plt.rcParams['ytick.labelsize'] = 14    # Font size for y-axis tick labels
plt.rcParams['legend.fontsize'] = 15    # Font size for legend


### CryoTriple4B4W4B4W4B4W model definition data and matrix element functions exported from Mathematica 

These are some fucntions that output details about the eigenmodes

Some baseline changes:
1. Length of the suspension wires - change to 0.298 m
2. Spacing between wires on the middle and optic stages set by the size of the ingot - \n
   n3 = 0.0405 \n
    n4 = n3 \n
    n5 = n3 

In [10]:
n3 = 0.0405
def reboot_params():
    importlib.reload(tp)
    tp.n3 = 0.0405
    tp.n4 = n3
    tp.n5 = n3
    revised_sus_length = 0.298
    tp.kw1= 1396.07
    tp.kw2=tp.kw1
    tp.kw3=70520.4
    tp.l1 = revised_sus_length
    tp.l2 = revised_sus_length
    tp.l3 = revised_sus_length
    
    
    
def original_params():
    importlib.reload(tp)
    revised_sus_length = 0.298



    data =     [["Masses", "m1 / m2 / m3", [round(tp.m1,2) ,round(tp.m2,2) ,round(tp.m3,2)]],

    ["Roll Moment of Inertia", "I1,x / I2,x / I3,x", [tp.I1x,tp.I2x,tp.I3x]],

    ["Pitch Moment of Inertia", "I1,y / I2,y / I3,y", [tp.I1y,tp.I2y,tp.I3y]],

    ["Yaw Moment of Inertia", "I1,z / I2,z / I3,z", [tp.I1z,tp.I2z,tp.I3z]],
    
            ["Stretched wire lengths", "l1 / l2 / l3", [tp.l1 , tp.l2, tp.l3]], 
            ["Upper end wire vertical attachment distance", "dtop / d1/ d3 " , [round(i,3) for i in [tp.dtop, tp.d1 , tp.d3]]],
            ["Bottom end wire vertical attachment distance", "d0 / d2 / d4" , [round(i,3) for i in [tp.d0 , tp.d2, tp.d4]]],
            
            ["Longitudinal wire separation" , "su / si / sl", [tp.su, tp.si , tp.sl]],
            ["Upper transverse wire separation", "n0 / n2 / n4 ", [tp.n0, tp.n2 , tp.n4]],
            ["Bottom transverse wire separation", "n1 / n3 / n5 ", [tp.n1, tp.n3 , tp.n5]],
            ["Vertical Spring stiffness", "kbuz / kbiz / kblz", [tp.kbuz, tp.kbiz, tp.kblz]],
            ["Wire Lengths", "l1 / l2 / l3", [tp.l1 , tp.l2 , tp.l3]],
            ["Youngs Moduli of the wires", "Y1 / Y2 / Y3", ['%.5E' % Decimal(tp.Y1), '%.5E' % Decimal(tp.Y2), '%.5E' % Decimal(tp.Y3)]]]



    params = pd.DataFrame(data, columns = ["Description", "Parameter","Values"])
    # Setting display options
    pd.set_option('display.width', 1000)  # Set width to a large value to prevent wrapping
    pd.set_option('display.max_colwidth', None)  # Display full contents of each cell without truncation
    print(params)
    
def print_modes(eigens):
    Hz = eigens[0]
    evecs = eigens[1]
    dofs = []
    dof_amps = []
    for j in range(0,18): 
        df =pd.DataFrame(np.array(list(zip(tpmodel.allvars,list(evecs[j])))), columns=["DOF","Amp"])
        df["Amp"] = [(round(float(i),2)) for i in list(df["Amp"])] 
        df = (df[df["Amp"]!=0])
        df = df.sort_values(by='Amp', ascending=False, key=abs)
        dofs.append(np.array(df["DOF"]))
        dof_amps.append(np.array(df["Amp"]))
        
    modes = pd.DataFrame({"Freq":Hz, "DOF":dofs, "Amps":dof_amps})
    
    def calculate_ratio(row):
        dof = row['DOF']
        amps = row['Amps']
        
        # Identify the mode type from the first entry
        mode = dof[0].rstrip('123')
        
        # Find indices of first and third occurrences for the identified mode
        first_idx = next((i for i, d in enumerate(dof) if d.startswith(mode + '1')), None)
        third_idx = next((i for i, d in enumerate(dof) if d.startswith(mode + '3')), None)
        
        # Calculate the ratio if both indices are found
        if first_idx is not None and third_idx is not None:
            return amps[first_idx] / amps[third_idx] if amps[third_idx] != 0 else None
        return None

    # Apply the function and create a new column
    modes['Mode_vis'] = modes.apply(calculate_ratio, axis=1)

# # Display the updated dataframe
# display(df)
    return modes
        
#         print (Hz[0])
def mode_forest(mode_amp, fig=None, ax = None):
    Hz = mode_amp[0]
    evecs = mode_amp[1]
    dofs = []
    dof_amps = []
    for j in range(0,18): 
        df =pd.DataFrame(np.array(list(zip(tpmodel.allvars,list(evecs[j])))), columns=["DOF","Amp"])
        df["Amp"] = [np.abs(round(float(i),2)) for i in list(df["Amp"])] 
        df = (df[df["Amp"]!=0])
        df = df.sort_values(by='Amp', ascending=False)
        dofs.append(np.array(df["DOF"]))
        dof_amps.append(np.array(df["Amp"]))
        
    
    
        if any(df["DOF"].iloc[i] in ["x1", "x2", "x3", "pitch1", "pitch2", "pitch3"] and df["Amp"].iloc[i] > 0.1 for i in range(len(df))):
            ax.scatter(Hz[j],"Long-pitch Mode", c= "red")
            ax.text(Hz[j] + 0.1,"Long-pitch Mode", str(round(Hz[j],3)), fontsize=10)  # Adjust 0.1 to position the text appropriately
        elif any(df["DOF"].iloc[j] in ["y1", "y2", "y3","roll1", "roll2", "roll3"] and df["Amp"].iloc[j] > 0.1 for j in range(len(df))):
            ax.scatter(Hz[j],"Trans-roll Mode", c = "blue")
            ax.text(Hz[j] + 0.1,"Trans-roll Mode", str(round(Hz[j],3)), fontsize=15)
        elif any(df["DOF"].iloc[j] in ["z1", "z2", "z3"] and df["Amp"].iloc[j] > 0.1 for j in range(len(df))):
            ax.scatter(Hz[j],"Bounce Mode", c = "black")    
            ax.text(Hz[j] + 0.1,"Bounce Mode", str(round(Hz[j],3)), fontsize=15)
        elif any(df["DOF"].iloc[j] in ["yaw1", "yaw2", "yaw3"] and df["Amp"].iloc[j] > 0.1 for j in range(len(df))):
            ax.scatter(Hz[j],"Yaw Mode", c = "green")    
            ax.text(Hz[j] + 0.1,"Yaw Mode", str(round(Hz[j],3)), fontsize=15)
#         elif any(df["DOF"].iloc[j] in ["pitch1", "pitch2", "pitch3"] and df["Amp"].iloc[j] > 0.5 for j in range(len(df))):
#             ax.scatter(Hz[j],"Pitch", c = "cyan")
#             ax.text(Hz[j] + 0.1,"Pitch", str(round(Hz[j],3)), fontsize=15)
#         elif any(df["DOF"].iloc[j] in ["roll1", "roll2", "roll3"] and df["Amp"].iloc[j] > 0.5 for j in range(len(df))):
#             ax.scatter(Hz[j],"Roll", c = "brown")
#             ax.text(Hz[j] + 0.1,"Roll", str(round(Hz[j],3)), fontsize=15)
        ax.grid()
        ax.set_ylabel("Modes", fontsize = 20)
        ax.set_xlabel("Freq (Hz)", fontsize = 20)
        ax.tick_params(axis = "both", labelsize = 15)
        ax.xaxis.set_minor_locator(MultipleLocator(10))
        
    modes = pd.DataFrame({"Freq":Hz, "DOF":dofs, "Amps":dof_amps})
        
    return fig, ax
         



def changes(mutation):
    reboot_params()
    for name, value in zip(mutation.iloc[:, 0], mutation.iloc[:, 1]):
        locals()[name] = value
    # print(mutation)
    # print(d2)
    revised_sus_length = 0.298
    tp.kw1= 1396.07
    tp.kw2=tp.kw1
    tp.kw3=70520.4
    tp.l1 = revised_sus_length
    tp.l2 = revised_sus_length
    tp.l3 = revised_sus_length
    #Some constraints 
    global n3
    n3 = 0.0405
    n4 = n3
    n5 = n3
    s = [su, si, sl]
    d = [d2, d3 , d4]
    
    if any(i>26.5e-3 for i in d)==True:
        print ("d is not practical")
    if any(i>28.5e-3 for i in s)== True:
        print ("s is not practical")
        
    def assign_to_tp(mutation, tp):
        for index, row in mutation.iterrows():
            string = row['Parameters']
            value = row['Values']
            # Dynamically set the attribute to the corresponding value
            setattr(tp, string, value)

# Call the function
    assign_to_tp(mutation, tp)
    # this seems to have a global imapct and i am not able to print out the original parameters again 
    # tp.d0 = d0  
    # tp.d1 = d1
    # tp.d2 = d2
    # tp.d3 = d3
    # tp.d4 = d4
    # tp.n0 = n0 
    # tp.n1 = n1 
    # tp.n2 = n2
    # tp.su = su
    # tp.si = si
    # tp.sl = sl
    # tp.n3 = n3
    # tp.n4 = n4
    # tp.n5 = n3 
    
    
    
    pend = tpmodel.pendSS(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)
    (Hz,evecs) = tpmodel.eigenmodes(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)
    mode_amp = (Hz,evecs)
    
    data =     [["Masses", "m1 / m2 / m3", [round(tp.m1,2) ,round(tp.m2,2) ,round(tp.m3,2)]],

    ["Roll Moment of Inertia", "I1,x / I2,x / I3,x", [tp.I1x,tp.I2x,tp.I3x]],

    ["Pitch Moment of Inertia", "I1,y / I2,y / I3,y", [tp.I1y,tp.I2y,tp.I3y]],

    ["Yaw Moment of Inertia", "I1,z / I2,z / I3,z", [tp.I1z,tp.I2z,tp.I3z]],
    
            ["Stretched wire lengths", "l1 / l2 / l3", [tp.l1 , tp.l2, tp.l3]], 
            ["Upper end wire vertical attachment distance", "dtop / d1/ d3 " , [round(i,3) for i in [tp.dtop, tp.d1 , tp.d3]]],
            ["Bottom end wire vertical attachment distance", "d0 / d2 / d4" , [round(i,3) for i in [tp.d0 , tp.d2, tp.d4]]],
            
            ["Longitudinal wire separation" , "su / si / sl", [tp.su, tp.si , tp.sl]],
            ["Upper transverse wire separation", "n0 / n2 / n4 ", [tp.n0, tp.n2 , tp.n4]],
            ["Bottom transverse wire separation", "n1 / n3 / n5 ", [tp.n1, tp.n3 , tp.n5]],
            ["Vertical Spring stiffness", "kbuz / kbiz / kblz", [tp.kbuz, tp.kbiz, tp.kblz]],
            ["Wire Lengths", "l1 / l2 / l3", [tp.l1 , tp.l2 , tp.l3]],
            ["Youngs Moduli of the wires", "Y1 / Y2 / Y3", ['%.5E' % Decimal(tp.Y1), '%.5E' % Decimal(tp.Y2), '%.5E' % Decimal(tp.Y3)]]]


    params = pd.DataFrame(data, columns = ["Description", "Parameter","Values"])
    # Setting display options
    pd.set_option('display.width', 1000)  # Set width to a large value to prevent wrapping
    pd.set_option('display.max_colwidth', None)  # Display full contents of each cell without truncation
    return params, pend, mode_amp

# def new_pend(d0,d1,d2,d3,d4,n0,n1,n2,su,si,sl):
#     #Some constraints 
#     global n3
#     n3 = 0.0405
#     n4 = n3
#     n5 = n3
#     d = [d2, d3 , d4]
#     s = [su, si, sl]
#     if any(i>26.5e-3 for i in d)==True:
#         print ("d is not practical")
#     if any(i>28.5e-3 for i in s)== True:
#         print ("s is not practical")
#     tp.d0 = d0  
#     tp.d1 = d1
#     tp.d2 = d2
#     tp.d3 = d3
#     tp.d4 = d4
#     tp.n0 = n0 
#     tp.n1 = n1 
#     tp.n2 = n2
#     tp.su = su
#     tp.si = si
#     tp.sl = sl
#     tp.n3 = n3
#     tp.n4 = n4
#     tp.n5 = n3 
    
    
#     return tpmodel.pendSS(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

def trimio_tf(pend,newinputs,newoutputs):
	'''A function to trim a state space object to have only the specified subsets of inputs and output.'''
	A=pend.A
	B=pend.B
	C=pend.C
	D=pend.D
	oldinputs=pend.input_index
	oldoutputs=pend.output_index
	oldstates=pend.state_index
	newinputlist = []
	for i in range(len(newinputs)):
			newinputlist.append(oldinputs.get(newinputs[i],0))
	newoutputlist = []
	for i in range(len(newoutputs)):
			newoutputlist.append(oldoutputs.get(newoutputs[i],0))
	BB=B[range(len(oldstates))][...,newinputlist]
	CC=C[newoutputlist]
	DD=D[newoutputlist][...,newinputlist]
	return ct.ss2tf(A,BB,CC,DD)


In [11]:
import symbexportCryoTriple4B4W4B4W4B4Wfull as tpmodel

### Parameters exported from Mathematica

NOTE: The "MatrixElementExport" part of the following line (i.e., between the underscores) is the Mathematica case name, and will need to be edited to use parameters from realistic cases.

In [14]:
import CryoTriple4B4W4B4W4B4W_MatrixElementExport_triplep as tp

Here the input d values are the wire physical attachment points and SS matrices are corrected for this while estiamting eigens

In [16]:
# 		dtop = dtop + flex1
# 		d0 = d0 + flex1
# 		d1 = d1 + flex2
# 		d2 = d2 + flex2
# 		d3 = d3 + flex3
# 		d4 = d4 + flex3


nw1 = 4
nw2 = 4
nw3 = 4
def flex():
    c1 = sqrt(tp.l1**2 - (tp.n1 - tp.n0)**2)/tp.l1
    c2 =  sqrt(tp.l2**2 - (tp.n3 - tp.n2)**2)/tp.l2
    c3 =  sqrt(tp.l3**2 - (tp.n5 - tp.n4)**2)/tp.l3
    flex1 = np.sqrt(nw1*tp.M11*tp.Y1/(tp.m1 + tp.m2 + tp.m3)/tp.g)*c1**(3/2)
    flex2 = np.sqrt(nw2*tp.M21*tp.Y2/(tp.m2 +tp.m3)/tp.g)*c2**(3/2)
    flex3 = np.sqrt(nw3*tp.M31*tp.Y3/tp.m3/tp.g)*c3**(3/2)
    return [flex1, flex2, flex3]

print (flex())

[9.2612824048435e-05, 0.00011532924934661046, 0.005232071157203804]


# SS Matrices

In [18]:
### The stiffness matrix for the floats

qm = tpmodel.qm(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

print ("stiffness matrix shape",qm.shape)

### The coupling matrix between floats and params

cqsm = tpmodel.cqsm(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

print ("shape of the coupling matrix between floats and params",cqsm.shape)

### The coupling matrix between floats and vars

cqxm = tpmodel.cqxm(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

print ("shape of the coupling matrix between floats and vars",cqxm.shape)


### The stiffness matrix for the params (raw, then effective)

sm = tpmodel.sm(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

sm.shape

smeff = tpmodel.smeff(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

smeff.shape

### The coupling matrix between vars and params (raw then effective)

cxsm = tpmodel.cxsm(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

cxsm.shape

cxsmeff = tpmodel.cxsmeff(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

cxsmeff.shape

### The stiffness matrix for vars (raw, then effective)

xm = tpmodel.xm(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

xm.shape

xmeff=tpmodel.xmeff(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

xmeff.shape

### The mass matrix

km = tpmodel.km(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

km.shape

# Generate state space

## Generate full A, B, C, D matrices

A = tpmodel.A(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

print (A.shape)

B = tpmodel.B(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)
print (B.shape)

C = tpmodel.C(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

print (C.shape)

D = tpmodel.D(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

print (D.shape)

## Generate full state space object

pend = tpmodel.pendSS(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)

stiffness matrix shape (12, 12)
shape of the coupling matrix between floats and params (12, 6)
shape of the coupling matrix between floats and vars (12, 18)
(36, 36)
(36, 24)
(24, 36)
(24, 24)


# Eigenmode stuff

In [20]:
sus_params = np.array([tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31])
modes= tpmodel.eigenmodes(*sus_params)
print_modes(modes)






,Freq,DOF,Amps,Mode_vis
0,9.856869,"[roll2, roll1, roll3]","[0.7, -0.64, -0.32]",2.000000
1,7.760272,"[z2, z1, z3]","[-0.78, 0.48, 0.4]",1.200000
2,7.136858,"[roll1, roll3, roll2]","[0.74, -0.56, 0.37]",-1.321429
3,5.338014,"[z1, z3, z2]","[-0.78, 0.58, -0.24]",-1.344828
4,3.326665,"[yaw1, yaw2, yaw3]","[0.74, -0.66, 0.15]",4.933333
5,3.029761,"[pitch1, pitch2, pitch3]","[-0.99, 0.12, -0.01]",99.000000
6,2.842177,"[roll3, roll2, roll1]","[-0.75, -0.59, -0.3]",0.400000
7,2.441187,"[pitch2, pitch3, x1, x2, pitch1, x3]","[-0.73, 0.64, 0.19, -0.14, 0.04, 0.03]",0.062500
8,2.436715,"[roll3, y1, roll2, y2, roll1, y3]","[0.61, 0.52, 0.41, -0.38, 0.18, 0.08]",0.295082
9,2.233000,"[pitch2, pitch3, pitch1]","[-0.97, 0.21, -0.12]",-0.571429


## Examine the lowest frequency mode (x and pitch)

# Original Sus parameters


In [23]:
original_params()
# fig, ax = plt.subplots(figsize = (20,10))
mode_amp = tpmodel.eigenmodes(tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31)
print_modes(mode_amp)





                                     Description           Parameter                                                                 Values
0                                         Masses        m1 / m2 / m3                                                        [1.1, 1.0, 1.0]
1                         Roll Moment of Inertia  I1,x / I2,x / I3,x  [0.0010787819175000002, 0.0011725228533440682, 0.0011725228533440682]
2                        Pitch Moment of Inertia  I1,y / I2,y / I3,y  [0.0008967735000000002, 0.0008647733010319863, 0.0008647733010319863]
3                          Yaw Moment of Inertia  I1,z / I2,z / I3,z  [0.0010787819175000002, 0.0008647733010319863, 0.0008647733010319863]
4                         Stretched wire lengths        l1 / l2 / l3                                                     [0.25, 0.25, 0.25]
5    Upper end wire vertical attachment distance      dtop / d1/ d3                                                     [0.0, 0.005, 0.003]
6   Bottom end wire 

,Freq,DOF,Amps,Mode_vis
0,9.856869,"[roll2, roll1, roll3]","[0.7, -0.64, -0.32]",2.000000
1,7.760272,"[z2, z1, z3]","[-0.78, 0.48, 0.4]",1.200000
2,7.136858,"[roll1, roll3, roll2]","[0.74, -0.56, 0.37]",-1.321429
3,5.338014,"[z1, z3, z2]","[-0.78, 0.58, -0.24]",-1.344828
4,3.326665,"[yaw1, yaw2, yaw3]","[0.74, -0.66, 0.15]",4.933333
5,3.029761,"[pitch1, pitch2, pitch3]","[-0.99, 0.12, -0.01]",99.000000
6,2.842177,"[roll3, roll2, roll1]","[-0.75, -0.59, -0.3]",0.400000
7,2.441187,"[pitch2, pitch3, x1, x2, pitch1, x3]","[-0.73, 0.64, 0.19, -0.14, 0.04, 0.03]",0.062500
8,2.436715,"[roll3, y1, roll2, y2, roll1, y3]","[0.61, 0.52, 0.41, -0.38, 0.18, 0.08]",0.295082
9,2.233000,"[pitch2, pitch3, pitch1]","[-0.97, 0.21, -0.12]",-0.571429


# Let's reduce cross couplings
Making the attachemnt points such that the flexing happens right at the line of COM of hte masses

In [25]:
Case1_Tr = pd.DataFrame(data={"Parameters":["d0","d1", "d2", "d3", "d4", "n0","n1", "n2", "su", "si", "sl"], "Values":[-flex()[0]+10e-7,-flex()[1],-flex()[1], -flex()[2],-flex()[2],n3+0.05, n3+0.01,n3, tp.su ,tp.si,tp.sl ]})
# Convert the "Parameters" column to column names and "Values" column to row values
Case1 = Case1_Tr.set_index("Parameters").T

display(Case1)
# Display the new DataFrame
# display(Case1_Tr)
# params1, pend1, modes1 = changes(Case1.loc["Values"].tolist())
params1, pend1, modes1 = changes(Case1_Tr)
# fig, ax = plt.subplots(figsize=(20,10))
# mode_forest(modes, fig, ax)
print (params1)
# ax.set_ylabel("Modes", fontsize = 20)
# ax.set_xlabel("Freq (Hz)", fontsize = 20)
# ax.tick_params(axis = "both", labelsize = 15)
df = print_modes(modes1)

display(df)

fig, ax = plt.subplots(figsize = (20,10))
mode_forest(modes1, fig, ax)



Parameters,d0,d1,d2,d3,d4,n0,n1,n2,su,si,sl
Values,-0.000092,-0.000115,-0.000115,-0.005232,-0.005232,0.0905,0.0505,0.0405,0.005,0.005,0.005


NameError: name 'd2' is not defined

## Mode visibility at the top mass due to cryocooler motion 

In [ ]:
tf_x0x1 = trimio_tf(pend,["x00"],["x1"])
tf_y0y1 = trimio_tf(pend,["y00"],["y1"])
mag_x0x1, _, omega = ct.frequency_response(tf_x0x1,omega_limits=[2*pi*0.1,2*pi*10], Hz = False)
mag_y0y1, _, omega = ct.frequency_response(tf_y0y1,omega_limits=[2*pi*0.1,2*pi*10], Hz = False)

LeidenData = pd.read_csv(r'/Users/cosmos1998/Desktop/Work/gla_phd/IGR/CryoCav/Vibrations CS110-2PT_FromLeiden.txt',names=['freq','vert','hor'],sep='\t')
freq = np.array(omega)/(2*np.pi)

fCryo = LeidenData.freq.values
vertCryo = LeidenData.vert.values
horCryo = LeidenData.hor.values

LongcryoInterp = np.interp(freq, fCryo, horCryo)
VertcryoInterp = np.interp(freq, fCryo, vertCryo)

TopMass_Modes_x = LongcryoInterp * mag_x0x1.transpose()
TopMass_Modes_z = VertcryoInterp * mag_y0y1.transpose()

fig,ax= plt.subplots(2,1,figsize = (10,10))
# 0.1 in the denominator corresponds to 
# ax.loglog(freq,MirrorLong_CryoVert*1e-6*np.sqrt(2),"-b", label="Length Noise due to vertical Cryocooler noise")
ax[0].loglog(freq,TopMass_Modes_x*1e-6,"-r", label="Top mass noise due to longitudinal Cryocooler noise - pend1")
ax[1].loglog(freq,TopMass_Modes_z*1e-6,"-k", label="Top mass noise due to vertical Cryocooler noise - pend1")
fig.text(0.5, 0.04, 'Freq', ha='center')  # X-axis label for all
fig.text(0.04, 0.5, 'm/rtHz', va='center', rotation='vertical')  # Y-axis label for all

# ax.loglog(freq,np.sqrt((MirrorLong_CryoLong*1e-6*np.sqrt(2))**2) ,"-k", label="Total Length Noise Cryocooler")
ax[0].legend()
ax[1].legend()
start, end = ax[0].get_ylim()
start, end = ax[1].get_ylim()
plt.vlines(100,start, end ,colors='black', linestyles='dashdot')

plt.show()

Observations:

All the transverse roll modes are mostly roll 
The long pitch mode freq is very low


## Using pend1 as baseline for investigation of what happens when certain pendulum parameters are altered

Infinte stiffness at the bottom stage. 

In [ ]:
display(Case1_Tr)
Case2_Tr= Case1_Tr.copy(deep = True)
Case2_Tr.loc[len(Case2_Tr)] = ['kblz',1] 

display(Case2_Tr)
params2, pend2, modes2 = changes(Case2_Tr)
# fig, ax = plt.subplots(figsize=(20,10))
# mode_forest(modes, fig, ax)
print (params2)
# ax.set_ylabel("Modes", fontsize = 20)
# ax.set_xlabel("Freq (Hz)", fontsize = 20)
# ax.tick_params(axis = "both", labelsize = 15)
df = print_modes(modes2)

display(df)

fig, ax = plt.subplots(figsize = (20,10))
mode_forest(modes2, fig, ax)



# Heavy top mass

In [ ]:
Case3_Tr= Case1_Tr.copy(deep = True)
Case3_Tr.loc[len(Case3_Tr)] = ["m1",5] 

display(Case3_Tr)
params3, pend3, modes3 = changes(Case3_Tr)
# fig, ax = plt.subplots(figsize=(20,10))
# mode_forest(modes, fig, ax)
print (params3)
# ax.set_ylabel("Modes", fontsize = 20)
# ax.set_xlabel("Freq (Hz)", fontsize = 20)
# ax.tick_params(axis = "both", labelsize = 15)
df = print_modes(modes3)

display(df)

fig, ax = plt.subplots(figsize = (20,10))
mode_forest(modes3, fig, ax)

The yaw mode visibity is low

In [ ]:
Case4_Tr= Case3_Tr.copy(deep = True)
Case4_Tr.loc[Case4_Tr['Parameters'] == 'n1', 'Values'] = 0.08
Case4_Tr.loc[Case4_Tr['Parameters'] == 'n2', 'Values'] = 0.08
Case4_Tr.loc[Case4_Tr['Parameters'] == 'm1', 'Values'] = 5
display(Case4_Tr)
params4, pend4, modes4 = changes(Case4_Tr)
# fig, ax = plt.subplots(figsize=(20,10))
# mode_forest(modes, fig, ax)
print (params4)
# ax.set_ylabel("Modes", fontsize = 20)
# ax.set_xlabel("Freq (Hz)", fontsize = 20)
# ax.tick_params(axis = "both", labelsize = 15)
df = print_modes(modes4)

display(df)

fig, ax = plt.subplots(figsize = (20,10))
mode_forest(modes4, fig, ax)

All these modes are soo closeby tho, this is not desirable as damping one could excite another mode. 

# Heavy Top Mass and d0 and d1 are 1 mm 

In [ ]:
Case5_Tr = Case3_Tr.copy(deep = True)
Case5_Tr.loc[Case5_Tr['Parameters'] == 'd0', 'Values'] = 10e-3
Case5_Tr.loc[Case5_Tr['Parameters'] == 'd1', 'Values'] = 10e-3
Case5_Tr.loc[Case5_Tr['Parameters'] == 'm1', 'Values'] = 1
display(Case5_Tr)
params5, pend5, modes5 = changes(Case5_Tr)
# fig, ax = plt.subplots(figsize=(20,10))
# mode_forest(modes, fig, ax)
print (params5)
# ax.set_ylabel("Modes", fontsize = 20)
# ax.set_xlabel("Freq (Hz)", fontsize = 20)
# ax.tick_params(axis = "both", labelsize = 15)
df = print_modes(modes5)

display(df)

fig, ax = plt.subplots(figsize = (20,10))
mode_forest(modes5, fig, ax)

In [ ]:
tf = trimio_tf(pend,["x3"],["x1"])
tf1 = trimio_tf(pend,["x3"],["x3"])
tf2 = trimio_tf(pend,["x3"],["p1"])
tf3 = trimio_tf(pend,["x3"],["p3"])

# Compute the frequency response of the transfer function H(jω)
mag, phase, omega = ct.frequency_response(tf,omega_limits=[2*pi*0.1,2*pi*1000], Hz = False)
mag1, phase1, omega = ct.frequency_response(tf1,omega_limits=[2*pi*0.1,2*pi*1000], Hz = False)
mag2, phase2, omega = ct.frequency_response(tf2,omega_limits=[2*pi*0.1,2*pi*1000], Hz = False)
mag3, phase3, omega = ct.frequency_response(tf3,omega_limits=[2*pi*0.1,2*pi*1000], Hz = False)

freq = np.array(omega)/(2*np.pi)

# Define input noise spectral density (constant, for white noise)
ThermalEasy = 1e-17*np.sqrt(100)/np.sqrt(freq)
S_n = ThermalEasy  # Example input noise spectral density (constant)

# Output spectral density S_y(f) = |H(jω)|^2 * S_n
S_y = (mag ** 2) * S_n
S_y1 = (mag1 ** 2) * S_n
S_y2 = (mag2 ** 2) * S_n
S_y3 = (mag3 ** 2) * S_n
RIN_flag_0 = pd.read_csv("/Users/cosmos1998/personal_thejas_svn/shadow_sens/011024/Rin_flag0_20241001_155016_Traces.csv",skiprows= np.linspace(0,11,11))
(RIN_flag_0.columns.values[0]) = "Freq"
(RIN_flag_0.columns.values[1]) = "TpABCD"
resp = 12e3 # V/m

# Plotting the output spectral density
fig,ax = plt.subplots(figsize = (10,5))
ax.loglog(RIN_flag_0["Freq"], RIN_flag_0["TpABCD"]/resp,label = "Sensor Noise")

ax.loglog(freq, S_y, label="x3-> x1")
ax.loglog(freq, S_y1,  label="x3-> x3")

# ax.loglog(freq, S_y2,  label="f_x3-> p1")
# ax.loglog(freq, S_y3,  label="x3-> p3")

ax.set_title("Modes excited due to thermal noise at the optic")
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Noise ASD (m/rtHz)")
ax.grid(which='both', axis='both')
ax.legend()
from scipy.signal import find_peaks
peaks,_ = find_peaks(S_y, height=0.01, prominence =0.01)
print (freq[peaks])
print (mag[peaks])

# Leiden seismic Noise data

In [ ]:
LeidenData = pd.read_csv(r'/Users/cosmos1998/Desktop/Work/gla_phd/IGR/CryoCav/Vibrations CS110-2PT_FromLeiden.txt',names=['freq','vert','hor'],sep='\t')

fCryo = LeidenData.freq.values
vertCryo = LeidenData.vert.values
horCryo = LeidenData.hor.values

fig,ax = plt.subplots(figsize = (20,10))
#ax.loglog(seis_freq,xSei*1e-6,'-b',label='X') #convert from um to m
#ax.loglog(seis_freq,ySei*1e-6,'-r',label='Y')
#ax.loglog(seis_freq,zSei*1e-6,'-m',label='Z')

ax.loglog(fCryo,vertCryo*1e-6,'-k',label='Leiden Vertical')
ax.loglog(fCryo,horCryo*1e-6,'-g',label='Leiden Horizontal')
ax.grid(True,which='both')
ax.legend()
ax.set_xlim(1,1000)
#ax.set_ylim(1e-12,1e-4)
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Displacement [m/rtHz]')
plt.show()

In [ ]:
print (sm)

In [ ]:
importlib.reload(tp)
print (tp.d3)

In [ ]:
print (changes1[0])

In [ ]:
py_param= np.array([tp.g,tp.m1,tp.m2,tp.m3,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2x,tp.I2y,tp.I2z,tp.I2xy,tp.I2yz,tp.I2zx,tp.I3x,tp.I3y,tp.I3z,tp.I3xy,tp.I3yz,tp.I3zx,tp.dtop,tp.d0,tp.d1,tp.d2,tp.d3,tp.d4,tp.n0,tp.n1,tp.n2,tp.n3,tp.n4,tp.n5,tp.su,tp.si,tp.sl,tp.kbuz,tp.kbiz,tp.kblz,tp.l1,tp.l2,tp.l3,tp.kw1,tp.kw2,tp.kw3,tp.Y1,tp.Y2,tp.Y3,tp.M11,tp.M21,tp.M31])



In [ ]:
import json

# Open the JSON file and load the list
with open("/Users/cosmos1998/list.json", "r") as file:
    list_data = json.load(file)

# Display the list
mt_param = np.array(list_data)


In [ ]:
print ([round(i,2) for i in py_param - mt_param])


In [ ]:
print (tp.kw3)
print (tp.kw2)

# Common Mode Rejection

In [ ]:
tf = trimio_tf(pend5,["x0"],["x3"])
mag, phase, omega = ct.frequency_response(tf,omega_limits=[2*pi*0.1,2*pi*1000], Hz = False)

freq = np.array(omega)/(2*np.pi)
LongcryoInterp = np.interp(freq, fCryo, horCryo)

MirrorLong_CryoLong_pend5 = LongcryoInterp * mag.transpose()
fig,ax= plt.subplots(figsize = (10,7))
# 0.1 in the denominator corresponds to 
# ax.loglog(freq,MirrorLong_CryoVert*1e-6*np.sqrt(2),"-b", label="Length Noise due to vertical Cryocooler noise")
ax.loglog(freq,MirrorLong_CryoLong_pend5*1e-6,"-r", label="Length Noise due to longitudinal Cryocooler noise - pend5")
# ax.loglog(freq,np.sqrt((MirrorLong_CryoLong*1e-6*np.sqrt(2))**2) ,"-k", label="Total Length Noise Cryocooler")
start, end = ax.get_ylim()
# plt.vlines(100,start, end ,colors='black', linestyles='dashdot')

# plt.show()

Case6_Tr = Case5_Tr.copy(deep = True)
Case6_Tr.loc[len(Case6_Tr)] = ["l1",0.31] 
Case6_Tr.loc[len(Case6_Tr)] = ["l2",0.31] 
display(Case6_Tr)


params6, pend6, modes6 = changes(Case6_Tr)
tf = trimio_tf(pend6,["x0"],["x3"])
mag, phase, omega = ct.frequency_response(tf,omega_limits=[2*pi*0.1,2*pi*1000], Hz = False)

freq = np.array(omega)/(2*np.pi)
LongcryoInterp = np.interp(freq, fCryo, horCryo)

MirrorLong_CryoLong_pend6 = LongcryoInterp * mag.transpose()
ax.loglog(freq,MirrorLong_CryoLong_pend6*1e-6,"-k", label="Length Noise due to longitudinal Cryocooler noise - pend6")
ax.legend()


In [ ]:
print (tp.l1)

In [ ]:
changes2 = pd.DataFrame(data={"Parameters":["d0","d1", "d2", "d3", "d4", "n0","n1", "n2", "su", "si", "sl"], "Values":[-flex()[0]+10e-7,-flex()[1],-flex()[1], -flex()[2],-flex()[2],n3+0.05, n3+0.01,n3, tp.su ,tp.si,tp.sl ]})

# Convert the "Parameters" column to column names and "Values" column to row values
changes1_Tr = changes1.set_index("Parameters").T

# Display the new DataFrame

params, pend1, modes = changes(changes1_Tr.loc["Values"].tolist())
# fig, ax = plt.subplots(figsize=(20,10))
# mode_forest(modes, fig, ax)
print (params)
# ax.set_ylabel("Modes", fontsize = 20)
# ax.set_xlabel("Freq (Hz)", fontsize = 20)
# ax.tick_params(axis = "both", labelsize = 15)
df = print_modes(modes)

display(df)

fig, ax = plt.subplots(figsize = (20,10))
mode_forest(modes, fig, ax)
display(changes1_Tr)


In [ ]:
print (tp.r1)